## Импорт библиотек

In [2]:
%tensorflow_version 2.x
import tensorflow as tf

from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, Bidirectional, Dropout
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import re
from random import uniform
from collections import defaultdict
import requests
import codecs
import random
import networkx as nx
import math

!pip install pymystem3==0.1.10
import json
from pymystem3 import Mystem

m = Mystem()

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## Обучение

In [0]:
## АЛГОРИТМ АХО-КОРАСИКА ------------

def graph_construction(list_of_words, sentence_len):
    '''
    Построение графа всех возможных разбиений.
    '''
    sentence_len += 2

    G=nx.MultiDiGraph()
    G.add_nodes_from(range(0, sentence_len))
    G.number_of_nodes()
    base_for_graph = []
    
    for i in range(0, len(list_of_words)) :
        base_for_graph.append((list_of_words[i][1], str(i) + '_word'))
        base_for_graph.append((str(i) + '_word', list_of_words[i][2]))

    G.add_edges_from(base_for_graph)

    return [[list_of_words[int(word.split('_')[0])][0] for word in word if type(word) != int] \
        for word in nx.all_simple_paths(G, 0, sentence_len - 2)]


def edits1(word):
    '''
    Функция поиска всех возможных ближайших слов
    (с наименьшим количеством перестановок)
    модель Левенштейна
    '''
    n = len(word)
    return list( [word[0:i]+word[i+1:] for i in range(n)] +                       # удаление символа
                [word[0:i]+word[i+1]+word[i]+word[i+2:] for i in range(n-1)] +   # транспозиция соседних символов
                [word[0:i]+c+word[i+1:] for i in range(n) for c in alphabet] +   # замена символа
                [word[0:i]+c+word[i:] for i in range(n+1) for c in alphabet])   # вставка лишнего символа


class AhoNode:
# Класс для построения дерева

    def __init__(self):
        self.goto = {}
        self.out = []
        self.fail = None


def aho_create_forest(patterns):
    '''
    Создать бор - дерево паттернов.
    '''
    root = AhoNode()

    for path in patterns:
        node = root
        for symbol in path:
            node = node.goto.setdefault(symbol, AhoNode())
        node.out.append(path)
    return root


def aho_create_statemachine(patterns):
    '''
    Непосредственно содание автомата Ахо-Корасика.
    '''
    root = aho_create_forest(patterns)
#     print(root[0])
    queue = []
    for node in root.goto.values():
        queue.append(node)
        node.fail = root

        
    while len(queue) > 0:
        rnode = queue.pop(0)

        for key, unode in rnode.goto.items():
            queue.append(unode)
            fnode = rnode.fail
            while fnode is not None and key not in fnode.goto:
                fnode = fnode.fail
            unode.fail = fnode.goto[key] if fnode else root
            unode.out += unode.fail.out

    return root


def aho_find_all(s, root):
    '''
    Находит все возможные подстроки из набора узлов в строке.
    '''
    founded_words = []
    node = root

    for i in range(len(s)):
        while node is not None and s[i] not in node.goto:
            node = node.fail
        if node is None:
            node = root
            continue
        node = node.goto[s[i]]

        for pattern in node.out:
             founded_words.append((pattern, i - len(pattern) + 1, i + 1))
    return founded_words

def my_algorithm_start(string_of_letters, root):
    '''
    Запуск алгоритма.
    '''
    return merge_all_lists([aho_find_all(new_string, root) for new_string in [string_of_letters]])
#     return aho_find_all(string_of_letters, root)

def merge_all_lists(lstlst):
    all_lists=[]
    for lst in lstlst:
        all_lists.extend(lst)
    return all_lists



## ФУНКЦИЯ ИСПРАВЛЕНИЯ ОШИБОК 

## ФУНКЦИИ ДЛЯ ПРЕДОБРАБОТКИ СТРОК ---------------

def test_data_sep(data):
  return data[1500 : -1500], np.hstack((data[:1500], data[-1500:]))


def is_in(list_, vall):
    return vall in list(map(lambda x:x[0], list_))


def max_val(mass):
  '''
  Возвращает индекс последнего вхождения максимального
  элемента в список.
  '''
  return next(i for i in range(len(mass)-1, -1, -1) if mass[i] == max(mass))


def nearby_check_2(words):
  '''
  Проверка существуют ли слова получаемые объединением с 
  соседними словами и сущесттвует ли такое слово само.
  '''
  len_ = len(words) + 1

  if words[0] != '':
    return [is_in(my_algorithm_start(''.join(words[:i]), root), ''.join(words[:i])) for i in range(1, len_)] + \
      [False for i in range(len_, 6)]
  else: 
    return [False]


def min_list(lists):
  min_l = []
  min_len =  math.inf

  for list_ in lists:
    if len(list_) < min_len:
      min_l = []
      min_len = len(list_)
      min_l.append(list_)
    elif len(list_) == min_len:
      min_l.append(list_)

  if len(min_l) == 1:
    return min_l[0]

  else:
    return answer_func([], min_l, [])


def answer_func(left_cont, list_of_options, right_cont):
  probabilities = []
  
  for sent in list_of_options:
    count = 0
    prob = 0
    sent_ = left_cont + sent + right_cont

    while len(sent_) > 0:
      paded_sent, sent_ = sent_[:12], sent_[12:]
      count += 1
      prob += model.predict(pad_sequences(np.array(tokenizer.texts_to_sequences([' '.join(paded_sent)])), maxlen = 12))

    probabilities.append(prob[0][0]/count)

  return list_of_options[max_val(probabilities)]


def correction(part_of_sent, j, count_of_words, index):
  '''
  Вызов алгоритма исправления.
  '''
  max_len = count_of_words - j + 2
  list_of_fragmentation = []
  n = 6
  # print('j = ', j)

  # print(part_of_sent)
  if index == 0:
    while n > 1 and list_of_fragmentation == []:
      # print('n = ', n)
      if j == 0:
        merged_s = ''.join(part_of_sent[0: (max_len - 2 if n - 1 > max_len else n - 1)])
        left_cont = []
        right_cont = part_of_sent[(max_len - 2 if n - 1 > max_len else n - 1):]

      elif j == 1:
        merged_s = ''.join(part_of_sent[0:(max_len - 1 if n > max_len else n)])
        left_cont = []
        right_cont = part_of_sent[(max_len - 1 if n > max_len else n):]

      else:
        merged_s = ''.join(part_of_sent[1:(max_len if n + 1 > max_len else n + 1)])
        left_cont = part_of_sent[:1]
        right_cont = part_of_sent[(max_len if n + 1 > max_len else n + 1):]
        # print(merged_s)

      list_of_sent = my_algorithm_start(merged_s, root)

      list_of_fragmentation = graph_construction(list_of_sent, len(merged_s))
      n -= 1

  else:
    if j == 0:
      merged_s = ''.join(part_of_sent[0:(max_len if index + 1 > max_len else index + 1)])
      left_cont = []
      right_cont = part_of_sent[(max_len if index + 1 > max_len else index + 1):]

    elif j == 1:
      merged_s = ''.join(part_of_sent[1:(max_len if index + 2 > max_len else index + 2)])
      left_cont = part_of_sent[:1]
      right_cont = part_of_sent[(max_len if index + 2 > max_len else index + 2):]

    else:
      merged_s = ''.join(part_of_sent[2:(max_len if index + 3 > max_len else index + 3)])
      left_cont = part_of_sent[:2]
      right_cont = part_of_sent[(max_len if index + 3 > max_len else index + 3):]

    list_of_sent = my_algorithm_start(merged_s, root)
    list_of_fragmentation = graph_construction(list_of_sent, len(merged_s))

  if list_of_fragmentation == []:
      return [], 0

  if index == 0:
    return left_cont + min_list(list_of_fragmentation), 5 - n
  
  else:
    return left_cont + answer_func(left_cont, list_of_fragmentation, right_cont), 0


def final_cor(text_ans, flag_dot):
  text_ans = re.sub(" - ", "-", text_ans)
  text_ans = re.sub("  ", " ", text_ans)
  text_ans = re.sub("\. \. \.", "...", text_ans)
  if flag_dot == 1:
    text_ans = text_ans[:-2]
  return text_ans


def get_splitters(sentence):
  '''
  Выделяет все небуквенные символы в предложениии.
  '''
  not_letters = ' '.join(re.findall(split_to_part, sentence)).split(' ')
  return [elem for elem in not_letters if elem != '']


def get_split_sent(sentence):
  '''
  Преобразует предложение в список фраз отделёных
  друг от друга разделителями.
  '''
  clear_sent = re.sub('><', '', re.sub('[^а-яё ]', '<>', sentence))
  return [elem for elem in clear_sent.split('<>') if elem != '']



## ЗАГРУЗКА ДАННЫХ --------------

def load_data():
  '''
  Загрузка данных для обучения.
  '''
  # НЕ С НАРЕЧИЯМИ
  word_list_1 = ['невысоко', 'немало', 'немного', 'неплохо', 'несильно', 'недорого', 'неслабо', 'нехорошо', 'непросто', 'нелегко', 'негде', 'незачем', 'некуда', 
                'неоткуда', 'невдалеке', 'невмоготу', 'невмочь', 'невпопад', 'невтерпеж', 'недаром', 'невесть', 'не высоко а', 'не мало а', 'не много а', 
                'не плохо а', 'не сильно а', 'не дорого а', 'не слабо а', 'не хорошо а', 'не просто а', 'не легко а', 'далеко не', 'вовсе не', 'отнюдь не', 
                'ничуть не', 'нисколько не', 'нисколько не', 'не вполне', 'не здесь', 'не очень', 'не полностью', 'не совсем','не там', 'не туда', 'несмотря на', 
                'невзирая на', 'не смотря', 'не взирая']


  # ОСТАЛЬНЫЕ СЛОЖНЫЕ СЛУЧАИ СЛИТНО-РАЗДЕЛЬНОГО НАПИСАНИЯ
  word_list_2 = ['в течение', 'в деле', 'в продолжение', 'в силу', 'в виде', 'в области', 'в связи', 'в смысле', 'по мере', 'по причине', 'в меру', 'за исключением',
                'в отличие', 'по поводу', 'в отношении', 'иметь в виду', 'ввиду', 'навстречу', 'вслед', 'внутри', 'вроде', 'вместо', 'вследствие', 'наподобие', 
                'насчёт', 'сверх', 'впросак', 'наобум', 'насмарку', 'взаперти', 'всмятку', 'испокон', 'взатяжку', 'вброд', 'вслух', 'при том', 'что бы', 'притом',
                'вовремя', 'вволю', 'напоказ', 'вверх', 'снизу', 'вперёд', 'вглубь', 'ввек', 'наверх', 'вплотную', 'наудалую', 'зачастую', 'наглухо', 'сгоряча', 
                'набело', 'заживо', 'вдвое', 'натрое', 'вовсю', 'почему', 'поэтому', 'затем', 'доныне', 'навсегда', 'задаром', 'вполголоса', 'вполсилы', 'во время', 
                'по двое', 'по трое', 'по одному', 'по этому', 'по тому', 'за тем', 'поэтому', 'потому', 'затем ', 'во всю ширь', 'без оглядки', 'до упора', 
                'с виду', 'под стать', 'чтобы', 'от того', 'оттого', 'при чём']


  # СЛОВАРЬ ВСЕХ ВОЗМОЖНЫХ СЛОВ (ruscorpora)
  !wget http://ruscorpora.ru/ngrams/1grams-3.zip
  !unzip 1grams-3.zip


  # СЛОВАРЬ ВСЕХ ВОЗМОЖНЫХ СЛОВ (opencorpora)
  !wget http://opencorpora.org/files/export/ngrams/unigrams.cyr.lc.zip
  !unzip unigrams.cyr.lc.zip


  # КОРПУС РАЗМЕЧЕННЫХ ПРЕДЛОЖЕНИЙ (opencorpora)
  !wget http://opencorpora.org/files/export/annot/annot.opcorpora.xml.zip
  !unzip annot.opcorpora.xml.zip

  return word_list_1, word_list_2



def download_sentances(list_of_words_in):
  '''
  Выгрузка всех предложений содержащих указанные в 
  аргументе функции слова.
  '''
  lst_in = []

  for word in list_of_words_in:
    download_link = 'https://kartaslov.ru/предложения-со-словом/' + word

    parsed_sait = requests.get(download_link)
    splited_data = parsed_sait.text.split('class="v2-sentence-box">\r\n            ')[1:]
    for sent in splited_data:
      
      lst_in.append(re.sub('[-—,.–!?\'\"…«»</b>&laquo&raquo;():]', '', sent.split("        ")[0]).lower())
    
  return lst_in



def tokenizing(whole_list, nn):
  '''
  Разбивает предложение 
  whole_list:
  '''
  all_tekenized_sent = []
  # tokenizer

  for sentence in whole_list:

      tokenized_sent = []
      
      word_counter = 5
      splited_sentence = sentence.split(" ")
      for word in splited_sentence[3:-1]:

          sent = []
          tmp_word_counter = word_counter - 5

          while tmp_word_counter - word_counter < nn - 5:

              if (tmp_word_counter < 0 or tmp_word_counter > len(splited_sentence) - 1) == False:
                  sent.append(splited_sentence[tmp_word_counter])

              tmp_word_counter += 1

          all_tekenized_sent.append(sent)
          word_counter += 1

  return all_tekenized_sent



## НАЧАЛЬНЫЕ ЗНАЧЕНИЯ ---------------

word_list_1, word_list_2 = load_data()

list_of_sent_ne = download_sentances(word_list_1)
list_of_sent_hard = download_sentances(word_list_2)

split_regex = re.compile(r'[.|!|?|…]')
list_of_wrong_sent = []

# wrong_text = open("/content/text2.txt", "r",encoding='utf-8', errors='ignore').read()
# wrong_text_list = wrong_text.split(' ')
# random.shuffle(wrong_text_list) 
# wrong_tok = re.sub('  ', ' ', ' '.join(wrong_text_list))

# for wrong_sentence in split_regex.split(wrong_tok):
#   list_of_wrong_sent.append(re.sub('[-—,.–!?\'\"…«»():]', '', wrong_sentence).lower())



## СОЗДАНИЕ АВТОМАТА -------------

NWORDS_df = pd.read_csv('1grams-3.txt', '\t', names = ["частота", "слово"])

NWORDS_df = NWORDS_df.loc[NWORDS_df["слово"].str.len() > 6]
NWORDS_1 = NWORDS_df["слово"].tolist()
NWORDS_1 = [word for word in NWORDS_1 if type(word) == str]
NWORDS_1 = [word for word in NWORDS_1 if re.findall('[A-Za-z0-9]', word) == []]


NWORDS_df = pd.read_csv('unigrams.cyr.lc', '\t', names = ["слово", "частота", "частота/милион"])
NWORDS_df = NWORDS_df.loc[~((NWORDS_df["слово"].str.len() == 1) & (NWORDS_df["частота"] < 4000))]
NWORDS_df = NWORDS_df.loc[~((NWORDS_df["слово"].str.len() == 2) & (NWORDS_df["частота"] < 400))]
NWORDS_2 = NWORDS_df["слово"].tolist()
NWORDS_2 = [word for word in NWORDS_2 if type(word) == str]
NWORDS_2 = [word for word in NWORDS_2 if re.findall('[A-Za-z0-9]', word) == []]

NWORDS = list(set(NWORDS_2 + NWORDS_1))

root = aho_create_statemachine(NWORDS)

print('\n\n///---------------- Автомат создан ----------------///\n\n')



## СОЗДАНИЕ ОБУЧАЮЩЕЙ ВЫБОРКИ -----------------

with codecs.open("annot.opcorpora.xml", "r",encoding='utf-8', errors='ignore') as file:
  list_of_sent = []
  for pack in file.read().split("\n"):
      try:
          sent = pack.split('<source>')[1]
          list_of_sent.append(re.sub('[-—,.–!?\'\"…«»():]', '', sent).lower()[:-9])
      except Exception:
          "wrong"


correct_ne_10 = tokenizing(list_of_sent_ne, 10)
correct_ne_5 = tokenizing(list_of_sent_ne, 5)
              
correct_hard_10 = tokenizing(list_of_sent_hard, 10)
correct_hard_5 = tokenizing(list_of_sent_hard, 5)



list_of_sent_ne_unc = []

for sent in list_of_sent_ne:
  list_of_sent_ne_unc.append(re.sub("  ", "", (re.sub(" не", " не ", sent))))

all_uncorrect = tokenizing(list_of_sent_ne_unc, 10)
uncorrect_ne_10 = [sent for sent in all_uncorrect if any(elem.startswith("не") for elem in sent[1:len(sent)-1])]

all_uncorrect = tokenizing(list_of_sent_ne_unc, 5)
uncorrect_ne_5 = [sent for sent in all_uncorrect if any(elem.startswith("не") for elem in sent[1:len(sent)-1])]



list_of_sent_hurd_unc = []
hard_pre = ['с', 'в', 'по', 'за', 'на', 'при', 'во', 'не']

for sent in list_of_sent_hard:
  for pre in hard_pre:
    sent = (re.sub("  ", "", (re.sub(" " + pre, " " + pre + " ", sent))))
  list_of_sent_hurd_unc.append(sent)

all_uncorrect = tokenizing(list_of_sent_hurd_unc, 10)
uncorrect_hard_10 = [sent for sent in all_uncorrect if any(any(elem.startswith(pre) for pre in hard_pre) for elem in sent[1:len(sent)-1])]

all_uncorrect = tokenizing(list_of_sent_hurd_unc, 5)
uncorrect_hard_5 = [sent for sent in all_uncorrect if any(any(elem.startswith(pre) for pre in hard_pre) for elem in sent[1:len(sent)-1])]



part_of_list_of_sent = list_of_sent[:12000]
part_of_list_of_sent_wrong = []

wr_list = []
for sent in list_of_sent[-12000:]:
  tmp_sent = sent.split(' ')
  random.shuffle(tmp_sent)
  part_of_list_of_sent_wrong.append(' '.join(tmp_sent))

correct = tokenizing(part_of_list_of_sent, 9)
uncorrect = tokenizing(part_of_list_of_sent_wrong, 9)

all_tekenized_wrong_sent = uncorrect + uncorrect_ne_10 + uncorrect_ne_5 + uncorrect_hard_10 + uncorrect_hard_5
all_tekenized_wrong_sent = np.array(all_tekenized_wrong_sent)

all_tekenized_correct_sent = correct + correct_ne_10 + correct_ne_5 + correct_hard_10 + correct_hard_5
all_tekenized_correct_sent = np.array(all_tekenized_correct_sent)
# whole_list = part_of_list_of_sent + list_of_sent_ne + list_of_sent_hard
# whole_list = np.array(whole_list)
# all_tekenized_sent = tokenizing(whole_list, 12) + tokenizing(list_of_sent[-30000:], 2) 

y_train_1 = np.ones(len(all_tekenized_correct_sent))
y_train_2 = np.zeros(len(all_tekenized_wrong_sent))

X_train = np.hstack((all_tekenized_correct_sent, all_tekenized_wrong_sent))
y_train = np.hstack((y_train_1, y_train_2))

assert len(y_train) == len(X_train)



## ПОДГОТВКА ОБУЧАЮЩЕЙ ВЫБОРКИ ДЛЯ ПОДАЧИ В МОДЕЛЬ ----------

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

tokens = tokenizer.texts_to_sequences(X_train)

X_train_tok = np.array(tokens)
X_train_tok = pad_sequences(X_train_tok, maxlen = 12)

size_for = X_train_tok.reshape(1, 12*len(X_train_tok))

size_of_dic = len(np.unique(size_for[0]))
size_of_dic += 1

# X_train_tok, X_test = test_data_sep(X_train_tok)
# y_train, y_test = test_data_sep(y_train)

assert len(X_train_tok) == len(y_train)



## ОБУЧЕНИЕ МОДЕЛИ -----------

model = Sequential()
model.add(Embedding(size_of_dic, 9, input_length = 12, dropout = 0.35))
model.add(LSTM(100, return_sequences=True, dropout_W = 0.65, dropout_U = 0.60))
model.add(LSTM(100, dropout=0.05))
model.add(Dense(256, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))


model.compile(optimizer='adam',
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])


history = model.fit(X_train_tok, y_train, batch_size = 512, epochs = 15, validation_split=0.15)

print('\n\n///---------------- Модель обучена ----------------///\n\n')

plt.plot(history.history['accuracy'], 
         label='Доля верных ответов на обучающем наборе')
plt.plot(history.history['val_accuracy'], 
         label='Доля верных ответов на проверочном наборе')
plt.xlabel('Эпоха обучения')
plt.ylabel('Доля верных ответов')
plt.legend()
plt.show()



# Исправление

In [22]:
text = open('/content/test_text.txt', 'r').read()


print(text)

text_ans = ''
split_to_sentece = re.compile(r'[.|!|?|…]')
split_to_part = (r'[\W\d]')
flag_dot = 0
if split_to_sentece.findall(text[-1]) == []:
  flag_dot = 1
  text += '.'

sentence_splitter = split_to_sentece.findall(text)

for sentence in split_to_sentece.split(text.lower())[:-1]:
  counter = 0
  not_letters = get_splitters(sentence)
  sent_splited = get_split_sent(sentence)
  # print(not_letters, sent_splited)

  lists_of_words = [[elem for elem in sent.split(' ') if elem != ''] for sent in sent_splited]
  words_in_sent = [elem for sent in sent_splited for elem in sent.split(' ') if elem != '']

  final = ''
  for i in range(len(sent_splited)):

    flag = 1
    jj = 0

    while flag != 0:
      flag = 0
      
      for j in range(jj, len(lists_of_words[i])):

        count_of_words = len(lists_of_words[i])
        # print(lists_of_words[i][j: j+5])
        val = nearby_check_2(lists_of_words[i][j: j+5])

        if (any(val[1:]) == True) or (val[0] == False):
          flag = 1

          if val[0] == False:
            index = 0
            # print(type(lists_of_words[i][j]))
            # statement = ('qual' in m.analyze(lists_of_words[i][j])[0]['analysis'][0].keys())
            statement = True
          else:
            index = max_val(val)

          if statement or index != 0:


            if i < len(sent_splited) - 1:
              dop = lists_of_words[i+1][:(0 if j+7-count_of_words<0 else j+7-count_of_words)]

            else:
              dop = []

            answer, n_val = correction(lists_of_words[i][(0 if j < 2 else j-2): j+7] + dop, j, count_of_words, index)
            # print('answer = ' ,answer, n_val)

            if answer != []:
              if index == 0:
                lists_of_words[i] = lists_of_words[i][:(0 if j < 2 else j-2)] + answer + lists_of_words[i][j+5-n_val:]
              else:
                lists_of_words[i] = lists_of_words[i][:(0 if j < 2 else j-2)] + answer + lists_of_words[i][j+index+1:]
                jj = j + 2
            else:
              flag = 0
          else:
              flag = 0
              jj = j + 2

          break
    
    sign = ''

    if len(not_letters) != 0:
      if not_letters[0] not in ',;:':
        sign = ' '
      sign = sign + not_letters[0]
      not_letters.remove(not_letters[0])

    str_ = ' '.join(lists_of_words[i])
    final = final + ' ' + str_ + sign


  text_ans = text_ans + final[1:].capitalize() + sentence_splitter[0] + ' '
  sentence_splitter.remove(sentence_splitter[0])

text_ans = final_cor(text_ans, flag_dot)

print(text_ans)

Это объясняется тем, что прои зошли большие из менения в жизни, в общественном и государственном устройстве, которыеоказаливлияниенаразвитие современного русского языка. Наиболее заметные проявления происходят в лексике: язык попо лн яет ся огр омным количеством заимствованных слов и оборотов, часто неоправданно. Однако существует и обратная сторона медали. Количество документов и другого контента резко растет — так, число элект ронных док ументов вотносите  льно крупной организации уже переваливает за миллионную отметку... С недавних пор вводится понятие электронного документооборота, так как объемы документов, производимых работниками умственного труда, стали измеряться не только папками, но и терабайтами. Отделымаркетингавыпускают пресс-релизы, отделы продаж составляют коммерческие предложения, юридические отделы штампуют договоры, отделы кадров формируют регламенты, высшее руководство издает приказы и трудится над новымист ратегическими презен  тациями. Документы — как бумажные, та